# Transformer-based Natural Language Processing
## Introduction to 🤗 Transformers
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/texttechnologylab/WiSe23-TFb-NLP/blob/master/assignment.ipynb)

### Installing necessary packages (i.e. if on Colab)

In [1]:
# Colab:
!pip install torch datasets tokenizers transformers

# Other:
# % pip install torch datasets tokenizers transformers

### Premise

This notebook will guide you through the process of finetuning a transformer model using the [🤗 Transformers](https://huggingface.co/docs/transformers/index) library.

First, we need to select a task and suitable dataset. Here, we will use the [Textual Entailment or Natrual Language Inference](https://cims.nyu.edu/~sbowman/multinli/) task as an example. A suitable dataset can be found in the [GLUE repository on the 🤗 Hub](https://huggingface.co/datasets/glue). The whole MNLI dataset ist way too big, so we will only use a slice of it.

We can load the MNLI (slice) of the GLUE dataset using [🤗 Datasets](https://huggingface.co/docs/datasets/index) as follows:

In [2]:
from datasets import load_dataset

mnli_dataset = load_dataset("glue", "mnli", split={"train": "train[:10%]", "validation": "validation_matched", "test": "test_matched"})
print(mnli_dataset)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

mnli/train-00000-of-00001.parquet:   0%|          | 0.00/52.2M [00:00<?, ?B/s]

mnli/validation_matched-00000-of-00001.p(…):   0%|          | 0.00/1.21M [00:00<?, ?B/s]

mnli/validation_mismatched-00000-of-0000(…):   0%|          | 0.00/1.25M [00:00<?, ?B/s]

...

Generating validation_matched split:   0%|          | 0/9815 [00:00<?, ? examples/s]

Generating validation_mismatched split:   0%|          | 0/9832 [00:00<?, ? examples/s]

Generating test_matched split:   0%|          | 0/9796 [00:00<?, ? examples/s]

Generating test_mismatched split:   0%|          | 0/9847 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 39270
    })
    validation: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9815
    })
    test: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9796
    })
})


As we can see above, the dataset is already split into train, development and test splits.
Each row contains four, but we only need to focus the premise, hypothesis and the label.

The textual entailment task requires us to recognize, given two text fragments, whether the meaning of one text is entailed (*can be inferred*) from the other text.

In this example, we will use a BERT-family model. With BERT, we formulate the entailment task as a simple classification task by concatenating the premise and hypothesis and training our classifier on the first token (the `[CLS]` token) of the input string:

```
"[CLS] This is the premise, i.e. a text that means something. [SEP] This is the hypothesis, i.e. what we may be able to infer [SEP]"
```

But let's first take a look at the dataset.

In [3]:
print(mnli_dataset['train'][:2])
print(mnli_dataset['validation'][:2])
print(mnli_dataset['test'][:2])

{'premise': ['Conceptually cream skimming has two basic dimensions - product and geography.', 'you know during the season and i guess at at your level uh you lose them to the next level if if they decide to recall the the parent team the Braves decide to call to recall a guy from triple A then a double A guy goes up to replace him and a single A guy goes up to replace him'], 'hypothesis': ['Product and geography are what make cream skimming work. ', 'You lose the things to the following level if the people recall.'], 'label': [1, 0], 'idx': [0, 1]}
{'premise': ['The new rights are nice enough', 'This site includes a list of all award winners and a searchable database of Government Executive articles.'], 'hypothesis': ['Everyone really likes the newest benefits ', 'The Government Executive articles housed on the website are not able to be searched.'], 'label': [1, 2], 'idx': [0, 1]}
{'premise': ['Hierbas, ans seco, ans dulce, and frigola are just a few names worth keeping a look-out for

As we see above, the `test_matched` split contains **unlabeled** samples, be we can ignore that for now.

Let's construct the sentences as we outlined above.

*Note:* The `[CLS]` and final `[SEP]` will be added by the BERT's tokenizer, so we omit them here.

In [25]:

from transformers import AutoTokenizer

def tokenize_mnli_data(data):
    return tokenizer(
        data['premise'],
        data['hypothesis'],
        truncation=True,
        padding=True
    )



In [26]:


prepared_dataset = mnli_dataset.map(
    lambda sample: {'input': f"{sample['premise']} [SEP] {sample['hypothesis']}"},

    remove_columns=['premise', 'hypothesis', 'idx']
)

#prepared_dataset = prepared_dataset.rename_column("label", "labels")
print(prepared_dataset['train'][:2])

{'label': [1, 0], 'input': ['Conceptually cream skimming has two basic dimensions - product and geography. [SEP] Product and geography are what make cream skimming work. ', 'you know during the season and i guess at at your level uh you lose them to the next level if if they decide to recall the the parent team the Braves decide to call to recall a guy from triple A then a double A guy goes up to replace him and a single A guy goes up to replace him [SEP] You lose the things to the following level if the people recall.']}


In [27]:
prepared_dataset = mnli_dataset.map(
    #lambda sample: {'input': f"{sample['premise']} [SEP] {sample['hypothesis']}",
                    #"label": sample["label"]},
    tokenize_mnli_data,
    batched=True,
    remove_columns=['premise', 'hypothesis', 'idx']
)

prepared_dataset = prepared_dataset.rename_column("label", "labels")
print(prepared_dataset['train'][:2])

Map:   0%|          | 0/39270 [00:00<?, ? examples/s]

Map:   0%|          | 0/9815 [00:00<?, ? examples/s]

Map:   0%|          | 0/9796 [00:00<?, ? examples/s]

{'labels': [1, 0], 'input_ids': [[101, 28103, 14795, 7081, 10458, 25004, 1144, 1160, 3501, 10082, 118, 3317, 1105, 14534, 119, 102, 22249, 1105, 14534, 1132, 1184, 1294, 7081, 10458, 25004, 1250, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 1128, 1221, 1219, 1103, 1265, 1105, 178, 3319, 1120, 1120, 1240, 1634, 14863, 1128, 3857, 1172, 1106, 1103, 1397, 1634, 1191, 1191, 1152, 4958, 1106, 9148, 1103, 1103, 6486, 1264, 1103, 14864, 4958, 1106, 1840, 1106, 9148, 170, 2564, 1121

In [28]:
print(prepared_dataset["train"][0].keys())

dict_keys(['labels', 'input_ids', 'token_type_ids', 'attention_mask'])


*Hint:* It is also possible to use the BERT tokenizer directly to construct the samples as shown above, skipping this preparation step entirely!

### Loading Pre-Trained Models

Now we need to load a pre-trained [BERT](https://github.com/google-research/bert) model. You should use a subclass of [AutoModel](https://huggingface.co/docs/transformers/main/en/autoclass_tutorial).

Viable pre-trained BERT models include:

<table>
<tr>
    <th>Model</th><th>Reference</th>
</tr>
<tr>
    <td><a href="https://huggingface.co/bert-base-uncased">bert-base-uncased</a></td>
    <td rowspan="6"><a href="https://aclanthology.org/N19-1423/">Devlin et al., 2019</a></td>
</tr>
<tr><td><a href="https://huggingface.co/bert-base-cased">bert-base-cased</a></td></tr>
<tr><td><a href="https://huggingface.co/bert-large-uncased">bert-large-uncased</a></td></tr>
<tr><td><a href="https://huggingface.co/bert-large-cased">bert-large-cased</a></td></tr>
<tr><td><a href="https://huggingface.co/bert-large-uncased-whole-word-masking">bert-large-uncased-whole-word-masking</a></td></tr>
<tr><td><a href="https://huggingface.co/bert-large-cased-whole-word-masking">bert-large-cased-whole-word-masking</a></td></tr>
<tr><td colspan="2"></td></tr>
<tr>
    <td><a href="https://huggingface.co/prajjwal1/bert-tiny">prajjwal1/bert-tiny</a></td>
    <td rowspan="4"><a href="https://arxiv.org/abs/1908.08962">Turc et al., 2019</a>; <a href="https://arxiv.org/abs/2110.01518">Bhargava et al., 2021</a></td>
</tr>
<tr><td><a href="https://huggingface.co/prajjwal1/bert-mini">prajjwal1/bert-mini</a></td></tr>
<tr><td><a href="https://huggingface.co/prajjwal1/bert-small">prajjwal1/bert-small</a></td></tr>
<tr><td><a href="https://huggingface.co/prajjwal1/bert-medium">prajjwal1/bert-medium</a></td></tr>
</table>


#### Load and instantiate a model for the textual entailment task

In [29]:
from transformers import AutoConfig, AutoTokenizer, AutoModelForSequenceClassification

config = AutoConfig.from_pretrained(
    "google-bert/bert-base-cased",
    num_labels=3
)

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")

model = AutoModelForSequenceClassification.from_pretrained(
    "google-bert/bert-base-cased",
    config=config
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Now we could use the [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) class for easy training. You can follow the tutorial from [the official documentation](https://huggingface.co/docs/transformers/quicktour#trainer-a-pytorch-optimized-training-loop).

#### Write the training procedure

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    train_dataset=prepared_dataset['train'],
    eval_dataset=prepared_dataset['validation'],
    processing_class=tokenizer,
    data_collator=data_collator,

)
trainer.train()

evaluation_result = trainer.evaluate(prepared_dataset["test"])
print(evaluation_result)
# TODO: evaluation

Step,Training Loss
500,0.994200


### Custom Training
While using the trainer class is very convenient, if you have to run custom procedures during training, a regular training loop can be more accessible.

We do need to do our own tokenization, though.

In [ ]:
def encode_pt(batch: dict):
    return tokenizer(
        batch['input'],
        add_special_tokens=True,
        return_token_type_ids=False,
        return_attention_mask=False,
        padding=False,
        truncation=True,
    )


pt_dataset = prepared_dataset.map(encode_pt)
print(pt_dataset['train'][:2])

However, in a manual training loop, we will want to make use of PyTorch's DataLoaders, which require some extra care to collate batches with samples of different lengths.

#### Implement `custom_collate`:
- Pad and stack the `input_ids` in a tensor.
- Stack the labels in a tensor of type `long`.

In [ ]:
import torch
from torch.nn.utils.rnn import pad_sequence


def custom_collate(batch: list[dict]) -> tuple[torch.Tensor, torch.Tensor]:
    input_ids = ...
    label = ...
    return input_ids, label

#### Write the training and evaluation loops

In [ ]:
from tqdm.notebook import tqdm, trange
from torch.utils.data import DataLoader
from torch.optim import *  # TODO
from torch.nn import *  # TODO

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

criterion = ...  # TODO
optimizer = ...  # TODO
num_epochs = ...  # TODO
batch_size = ...  # TODO

train_dataloader = DataLoader(pt_dataset['train'], batch_size=batch_size, shuffle=True, collate_fn=custom_collate)
dev_dataloader = DataLoader(pt_dataset['validation'], batch_size=batch_size, shuffle=False, collate_fn=custom_collate)

model.to(device)
for epoch in trange(num_epochs, position=0):
    model.train()
    for batch in tqdm(train_dataloader, position=1, leave=False):
        ...  # TODO

    model.eval()
    for batch in tqdm(dev_dataloader, position=1, leave=False):
        ...  # TODO